### 3.4 模型构建

####  3.4.1 神经网络的构造

In [1]:
import torch
from torch import nn

class MLP(nn.Module):
  def __init__(self, **kwargs):
    super(MLP, self).__init__()
    self.hidden = nn.Linear(784, 256)
    self.act = nn.ReLU()
    self.output = nn.Linear(256, 10)
    
  def forward(self, x):
    a = self.act(self.hidden(x))
    return self.output(a)

In [3]:
X = torch.rand(2, 784)
net = MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.0658,  0.1477,  0.0087, -0.1591, -0.0906,  0.1742,  0.0189,  0.1231,
          0.0275, -0.0591],
        [-0.0126,  0.0261,  0.0054, -0.0588, -0.0947,  0.1191,  0.0598, -0.0363,
          0.1346, -0.0306]], grad_fn=<AddmmBackward0>)

#### 3.4.2 神经网络中常见的层


1. 不含模型参数的层

In [7]:
import torch
from torch import nn

class MyLayer(nn.Module):
  def __init__(self, **kwargs):
    super(MyLayer, self).__init__(**kwargs)
  def forward(self, x):
    return x - x.mean()

In [8]:
layer = MyLayer()
layer(torch.tensor([1, 2, 3, 4, 5], dtype=torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

2. 含模型参数的层

In [15]:
class MyListDense(nn.Module):
  def __init__(self):
    super(MyListDense, self).__init__()
    self.params = nn.ParameterList([nn.Parameter(torch.randn(4, 4)) for i in range(3)])
    self.params.append(nn.Parameter(torch.randn(4, 1)))
    
  def forward(self, x):
    for i in range(len(self.params)):
      x = torch.mm(x, self.params[i])
    return x
  
net = MyListDense()
print(net)
net(torch.randn(4,4))

MyListDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.float32 of size 4x4]
      (1): Parameter containing: [torch.float32 of size 4x4]
      (2): Parameter containing: [torch.float32 of size 4x4]
      (3): Parameter containing: [torch.float32 of size 4x1]
  )
)


tensor([[-70.6800],
        [  8.7415],
        [ -3.3696],
        [ -0.9187]], grad_fn=<MmBackward0>)

In [17]:
class MyDictDense(nn.Module):
  def __init__(self):
    super(MyDictDense, self).__init__()
    self.params = nn.ParameterDict({
      'linear1': nn.Parameter(torch.randn(4, 4)),
      'linear2': nn.Parameter(torch.randn(4, 4)),
    })
    self.params.update({'linear3': nn.Parameter(torch.randn(4, 1))})
  
  def forward(self, x, choice='linear1'):
    return torch.mm(x, self.params[choice])
  
net = MyDictDense()
print(net)
net(torch.randn(4,4), "linear3")

MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)


tensor([[ 0.2645],
        [ 1.2620],
        [ 3.0418],
        [-0.0383]], grad_fn=<MmBackward0>)

3. 二维卷积层

In [ ]:
import torch
from torch import nn

def corr2d(X, K):
  h, w = K.shape
  X, K = X.float(), K.float()
  Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
  for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
      Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
  return Y

class Conv2D(nn.Module):
  def __init__(self, kernel_size):
    super(Conv2D, self).__init__()
    self.weight = nn.Parameter(torch.randn(kernel_size))
    self.bias = nn.Parameter(torch.randn(1))
    
  def forward(self, x):
    return corr2d(x, self.weight) + self.bias
  
